In [1]:
import pandas as pd

df = pd.read_csv('imdb.csv',index_col=0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### replce \N and NaN

In [2]:
df = df.replace(r'\\N','unknown', regex=True) 

In [3]:
df = df.fillna('unknown')

In [4]:
df.head(10)

,tconst,primaryTitle,startYear,runtimeMinutes,genres,directors,writers,actors,isAdult,averageRating,numVotes
0,tt0000009,Miss Jerry,1894,45,Romance,nm0085156,nm0085156,"nm0183823,nm1309758",0,5.4,108
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",nm0714557,unknown,unknown,0,5.2,341
2,tt0000335,Soldiers of the Cross,1900,unknown,"Biography,Drama","nm0095714,nm0675140",unknown,"nm1012612,nm1011210,nm1012621,nm0675239,nm0675260",0,6.1,40
3,tt0000502,Bohemios,1905,100,unknown,nm0063413,"nm0063413,nm0657268,nm0675388","nm0215752,nm0252720",0,4.4,5
4,tt0000574,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",nm0846879,nm0846879,"nm0846894,nm3002376",0,6.1,567
5,tt0000615,Robbery Under Arms,1907,unknown,Drama,nm0533958,"nm0092809,nm0533958","nm3071427,nm0581353,nm0888988,nm0240418,nm0346387",0,4.5,15
6,tt0000630,Hamlet,1908,unknown,Drama,nm0143333,nm0000636,unknown,0,3.4,11
7,tt0000675,Don Quijote,1908,unknown,Drama,nm0194088,nm0148859,unknown,0,4.8,10
8,tt0000676,Don Álvaro o la fuerza del sino,1908,unknown,Drama,nm0194088,nm0729044,"nm0097421,nm0140054",0,4.3,9
9,tt0000679,The Fairylogue and Radio-Plays,1908,120,"Adventure,Fantasy","nm0091767,nm0877783","nm0000875,nm0877783","nm0000875,nm0122665,nm0933446,nm2924919",0,4.8,34


#### IMDB's weighted rating formula

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where,

v is the number of votes for the movie     
m is the minimum votes required to be listed in the chart     
R is the average rating of the movie     
C is the mean vote across the whole report     

In [6]:
# Claculation of c 
numVotes = df[df['numVotes'].notnull()]['numVotes'].astype('int')
averageRating = df[df['averageRating'].notnull()]['averageRating'].astype('int')
C = averageRating.mean()
C

5.715438203065863

In [7]:
# Claculation of m
m = numVotes.quantile(0.95)
m

5129.0

In [8]:
def weighted_rating(x):
    v = x['numVotes']
    R = x['averageRating']
    return (v/(v+m) * R) + (m/(m+v) * C)

#### create keywords

In [9]:
df['keywords'] = df['genres'] + ',' + df['directors'] + ',' + df['writers'] + ',' + df['actors']

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df['keywords'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [23]:
df = df.reset_index()
titles = df['title']
indices = pd.Series(df.index, index=df['title'])

In [24]:
df

,index,adult,belongs_to_collection,genres,id,imdb_id,popularity,production_companies,production_countries,release_date,...,title,vote_average,vote_count,cast,crew,keywords,cast_size,crew_size,director,soup
0,0,False,Toy Story Collection,"[Animation, Comedy, Family]",862,tt0114709,21.9469,[Pixar Animation Studios],[United States of America],1995-10-30,...,Toy Story,7.7,5415.0,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousi, toy, boy, friendship, friend, rival...",13,106,"[johnlasseter, johnlasseter]",jealousi toy boy friendship friend rivalri boy...
1,1,False,[],"[Adventure, Fantasy, Family]",8844,tt0113497,17.0155,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,Jumanji,6.9,2413.0,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgam, disappear, basedonchildren'sbook, n...",26,16,"[joejohnston, joejohnston]",boardgam disappear basedonchildren'sbook newho...
2,2,False,Grumpy Old Men Collection,"[Romance, Comedy]",15602,tt0113228,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,Grumpier Old Men,6.5,92.0,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fish, bestfriend, duringcreditssting, oldmen]",7,4,"[howarddeutch, howarddeutch]",fish bestfriend duringcreditssting oldmen walt...
3,3,False,[],"[Comedy, Drama, Romance]",31357,tt0114885,3.85949,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,Waiting to Exhale,6.1,34.0,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",10,10,"[forestwhitaker, forestwhitaker]",basedonnovel interracialrelationship singlemot...
4,4,False,Father of the Bride Collection,[Comedy],11862,tt0113041,8.38752,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,Father of the Bride Part II,5.7,173.0,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[babi, midlifecrisi, confid, age, daughter, mo...",12,7,"[charlesshyer, charlesshyer]",babi midlifecrisi confid age daughter motherda...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46623,46623,False,[],"[Drama, Family]",439050,tt6209470,0.072051,[],[Iran],NaN,...,Subdue,4.0,1.0,"[leilahatami, kouroshtahami, elhamkorda]","[{'credit_id': '5894a97d925141426c00818c', 'de...",[tragiclov],3,9,"[hamidnematollah, hamidnematollah]",tragiclov leilahatami kouroshtahami elhamkorda...
46624,46624,False,[],[Drama],111109,tt2028550,0.178241,[Sine Olivia],[Philippines],2011-11-17,...,Century of Birthing,9.0,3.0,"[angelaquino, perrydizon, hazelorencio]","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...","[artist, play, pinoy]",11,6,"[lavdiaz, lavdiaz]",artist play pinoy angelaquino perrydizon hazel...
46625,46625,False,[],"[Action, Drama, Thriller]",67758,tt0303758,0.903007,[American World Pictures],[United States of America],2003-08-01,...,Betrayal,3.8,6.0,"[erikaeleniak, adambaldwin, juliedupage]","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",[],15,5,"[markl.lester, markl.lester]",erikaeleniak adambaldwin juliedupage markl.les...
46626,46626,False,[],[],227506,tt0008536,0.003503,[Yermoliev],[Russia],1917-10-21,...,Satan Triumphant,0.0,0.0,"[iwanmosschuchin, nathalielissenko, pavelpavlov]","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",[],5,2,"[yakovprotazanov, yakovprotazanov]",iwanmosschuchin nathalielissenko pavelpavlov y...


### Content Based Prediction

In [34]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = df.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [38]:
predict = improved_recommendations('The Godfather')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [43]:
predic_score = sum(predict['wr'])/len(predict['wr'])
predic_score

5.342363032965511